In [22]:
import os
import os.path as osp
import time
from collections import defaultdict
from typing import DefaultDict, Dict, Set, TextIO, Tuple, Union, List
import numpy as np
import sys
from abc import *
np.set_printoptions(threshold=sys.maxsize)

In [28]:
class DiGraph(metaclass=ABCMeta):
    
    @abstractmethod
    def __str__(self):
        pass
    
    @abstractmethod
    def nodes(self) -> Set[int]:
        pass
    
    @abstractmethod
    def edges(self) -> List[Tuple[int, int]]:
        pass
    
    @abstractmethod
    def add_vertex(self, v: int) -> None:
        pass
    
    @abstractmethod
    def remove_vertex(self, v: int) -> None:
        pass

    @abstractmethod
    def add_edge(self, src: int, dst: int) -> None:
        pass

    @abstractmethod
    def remove_edge(self, src: int, dst: int) -> None:
        pass

    @abstractmethod
    def out_neighbor(self, v: int):
        pass

    @abstractmethod
    def out_degree(self, v: int) -> int:
        pass

    @abstractmethod
    def has_edge(self, src: int, dst: int) -> bool:
        pass
    
    

In [30]:
class Graph_dict(DiGraph):
    def __init__(self) -> None:
        self._nodes = set()
        self._out_neighbor = defaultdict(set)
    
    def __str__(self):
        result = ""
        for v in self._nodes:
            neighbors = self._out_neighbor[v]
            if not neighbors:
                result = result + str(v) + " : empty\n"
            else : result = result + str(v) + str(neighbors) + '\n'
        return result
    
    def nodes(self) -> Set[int]:
        return self._nodes
    
    def edges(self) -> List[Tuple[int, int]]:
        edges = []
        for v1 in self._nodes:
            neighbors = self._out_neighbor[v1]
            for v2 in neighbors:
                edges.append((v1, v2))
        return edges
    
    def add_vertex(self, v: int) -> None:
        self._nodes.add(v)
    
    def remove_vertex(self, v: int) -> None:
        self._nodes.remove(v)
        for j in self._out_neighbor:
            if v in self._out_neighbor[j]:
                self._out_neighbor[j].remove(v)
        del(self._out_neighbor[v])
    
    def add_edge(self, src: int, dst: int) -> None:
        self._nodes.add(src)
        self._nodes.add(dst)
        self._out_neighbor[src] = dst
    
    def remove_edge(self, src: int, dst: int) -> None:
        self._out_neighbor[src].remove(dst)
        if not self._out_neighbor[src]:
            self._nodes.remove(src)

    def out_neighbor(self):
        return self._out_neighbor
    
    def out_degree(self, v: int) -> int:
        return len(self._out_neighbor[v])
    
    def has_edge(self, src: int, dst: int) -> bool:
        return dst in self._out_neighbor[src]
    
    def subgraph(self, vertices: List[int]):
        subgraph = Graph_dict()
        for v in vertices:
            subgraph.add_vertex(v)
        for v1 in vertices:
            for v2 in vertices:
                if v1 == v2:
                    continue
                elif self.has_edge(v1, v2):
                    subgraph.add_edge(v1, v2)
        return subgraph
          

In [37]:
class Graph_numpy_array(DiGraph):
    def __init__(self, size):
        self._nodes = set()
        self._data = np.zeros((size,size))
        self._max_size = size
        
    def __str__(self):
        return str(self._data)
    
    def nodes(self):
        return self._nodes

    def edges(self):
        edge_list = []
        for v1 in range(self._max_size):
            for v2 in range(self._max_size):
                if self._data[v1, v2] == 1:
                    edge_list.append((v1,v2))
        return edge_list
    
    def add_vertex(self,v: int) -> None:
        if v >= 0 or v < self._max_size:
            self._nodes.add(v)
        else:
            print("Error : Vertex is out of range")
    
    def remove_vertex(self, v: int) -> None:
        if v >= 0 or v < self._max_size:
            self._nodes.remove(v)
            self._data[:,v] = 0
            self._data[v,:] = 0
        else:
            print("Error : Vertex is out of range")
    
    def remove_edge(self, src: int, dst: int):
        pass
    
    def subgraph(self, vertices: List[int]):
        subgraph = Graph_numpy_array(10)
        for v in vertices:
            self._nodes.add(v)
        for v1 in range(self._max_size):
            for v2 in range(self._max_size):
                if v1 == v2:
                    continue
                else:
                    subgraph.add_edge(v1, v2)
        return subgraph

In [38]:
def l1_distance(x: DefaultDict[int, float], y: DefaultDict[int, float]) -> float:
    sum_x_y = 0
    for i in x.keys():
        sum_x_y += abs(x[i]-y[i])
    return sum_x_y

def pagerank(graph, maxiters, damping_factor, tol):
    vec = defaultdict(lambda :0)
    nodes_len = len(graph.nodes())
    for i in graph.nodes():
        vec[i] = 1/nodes_len
    for iter in range(maxiters):
        vec_new = defaultdict()
        S = 0
        for j in graph.nodes():
            neighbor_in = []
            for k in graph.edges():
                if j == k[1]:
                    neighbor_in.append(k[0])
            for i in neighbor_in:
                vec_new[j] += damping_factor*vec[i]/graph.out_degree(i)
            S += vec_new[j]
        for j in graph.nodes():
            vec_new[j] += (1-S)/nodes_len
        if l1_distance(vec, vec_new)<tol:
            break
        vec = vec_new
    return vec
            